# Demo for using `psmilu4py` #
In this example, we show how to use `psmilu4py` MILU preconditioner coupling with native GMRES solver from `scipy` package. The example is from a 2D poisson equation on unit disk with quadratic FEM.

In [1]:
try:
    from scipy.sparse.linalg import gmres
except ImportError:
    print('You need to have scipy installed!')
    raise
from psmilu4py import *
from psmilu4py.scipy import *
import numpy as np

The matrix is stored by the PSMILU native binary format that is leading symmetric block aware. It's worht noting that, unlike typical FEM, our test matrix was assembled in a way that all Dirichlet nodes are defered and not been eliminated. Therefore, we ended up with an PS system.

In [2]:
# load matrix and leading block size m
A, m = read_native_psmilu('../demo_inputs/A.psmilu')

In [3]:
# A is scipy.sparse.csr_matrix
print('A\'s type is ', type(A), '.\nleading symmetric block ', m, '.\nsize(A)=', A.shape)

A's type is  <class 'scipy.sparse.csr.csr_matrix'> .
leading symmetric block  387 .
size(A)= (2955, 2955)


The rhs vector can be directly loaded from `numpy` ASCII routine

In [4]:
b = np.loadtxt('../demo_inputs/b.txt')

In [5]:
assert A.shape[0] == len(b)

Now, we will create the PSMILU object for scipy, i.e. `ScipyPSMILU`. This object is a child of `LinearOperator`, which can be used as the preconditioner $\boldsymbol{M}^{-1}$.

In [6]:
M = ScipyPSMILU(A.shape)
M.factorize(A, m=m)  # factorize A with leading symmetric block size m


|    Pre-dominantly Symmetric Multi-level Incomplete LU (PS-MILU)     |
|                                                                     |
| PSMILU is a package for computing multi-level incomplete LU factor- |
| ization with optimal time complexity. PSMILU is the first software  |
| package to utilize the pre-dominantly symmetric systems, which occ- |
| ur quite often but were not precisely defined and appreciated.      |
|                                                                     |
-----------------------------------------------------------------------

 Package information:

		Copyright (C) The PSMILU AUTHORS
		Version: 0.0.0
		Built on: 14:12:47, Mar 18 2019


Options (control parameters) are:

tau_L                         0.010000
tau_U                         0.010000
tau_d                         10.000000
tau_kappa                     100.000000
alpha_L                       4
alpha_U                       4
rho                           0.250000
c_d           

Now, we can solve the for $\boldsymbol{x}=\boldsymbol{A}^{-1}\boldsymbol{b}$ with our MILU preconditioner M.
**Be aware that scipy uses left preconditioner, which is not optimal for PSMILU package**.

In [7]:
# Anyone, seriously, knows a more elegant way to report number of iterations??
it = 0
def report_iters(_):
    global it
    it += 1
# solve, should be intuitive
x, flag = gmres(A, b, tol=1e-6, M=M, callback=report_iters)

In [8]:
assert flag==0, 'failed to solve for x'

In [9]:
print('GMRES with left MILU took ', it, ' iterations to converge..')

GMRES with left MILU took  10  iterations to converge..


In [10]:
# post processing
x_ref = np.loadtxt('../demo_inputs/ref_x.txt')
err = x-x_ref
print('relative error is ', np.linalg.norm(err)/np.linalg.norm(x_ref))

relative error is  2.7502839845420405e-08
